In [1]:
#Import
from spotify import Client, OAuth
import pandas as pd
import pygn
import pickle

In [2]:
#Grace Note import
clientID = '740529234-9D358964C751528ED8F5DA9B0D060C84' # Enter your Client ID here
userID = pygn.register(clientID)

In [3]:
auth = OAuth('f716b10140c04e9997c89e1d094745f6', 'b9aa1c134fe244ebaed7b3cba541f6a7')
auth.request_client_credentials()

In [4]:
spotifyClient = Client(auth)

In [5]:
spotifyAPI = spotifyClient.api

In [6]:
def getTrackAttributes(trackIdList = []):
    print(trackIdList)
    trackJson = spotifyAPI.tracks_audio_features(trackIdList)
    trackList = trackJson['audio_features']
    trackList = filter(None, trackList)
    trackList = [x for x in trackList if x is not None]
    return trackList

In [7]:
def getAttributeDF(trackFeatList):
    trackDF = pd.DataFrame(trackFeatList)
    trackDF = trackDF.drop(['analysis_url', 'track_href', 'type', 'uri'], axis=1)
    return trackDF

In [8]:
def getPlaylistTrack(play_id,user_id):
    tracks = spotifyAPI.user_playlist_tracks(playlist_id= play_id, user_id= user_id,fields='items(track.id,track.album.name,track.name,track.artists)')
    #tracks = list(filter(None, tracks))
    return tracks


In [9]:
def trackJSONtoDF(tracks,genre):
    flatData = []

    for i in range(len(tracks['items'])):
        trackObj = {}
        
        trackObj['id'] = tracks['items'][i]['track']['id']
        trackObj['album'] = tracks['items'][i]['track']['album']['name']
        trackObj['track_name'] = tracks['items'][i]['track']['name']
        trackObj['artist'] = tracks['items'][i]['track']['artists'][0]['name']
        trackObj['genre'] = genre
        if trackObj['id'] != None:
            flatData.append(trackObj)
    return pd.DataFrame(flatData)

In [10]:
def pipeLineFunc(play_id,user_id,genre):
    # Fetch playlist tracks
    plTracks = getPlaylistTrack(play_id,user_id)
    
    
    # Convert to DF
    plTracksDF =  trackJSONtoDF(plTracks,genre)

    # Get list of track ids
    trackIds = list(plTracksDF['id'])

    # Get tracks features
    trackFeat = getTrackAttributes(trackIds)

    # Convert JSON to DF
    trackFeatDF = getAttributeDF(trackFeat)

    # Merging 2 data frames
    resultDF = pd.concat([plTracksDF, trackFeatDF], axis=1, join='inner')

    # Droping duplicate ID columns
    resultDF = resultDF.T.drop_duplicates().T

    return resultDF
    
    

In [ ]:
res = pipeLineFunc('2kpoUUJ5a4Cw3feTkFJhZ2','1276640268','top100')
res

In [ ]:
res.to_csv('billboardTop100.csv')

In [ ]:
res.columns

In [ ]:
playListData = [{'user':'spotify','playlist_id':'37i9dQZF1DWTwnEm1IYyoj','genre':'Pop'},
{'user':'1154148597','playlist_id':'03Wky4J3HehgLEqgkYmia6','genre':'Pop'},
{'user':'spotify','playlist_id':'37i9dQZF1DX5CHqBODmn2c','genre':'Pop'},
{'user':'1123398699','playlist_id':'23o2085ZcSb5V3yVxnHj9c','genre':'Pop'},
{'user':'1180623169','playlist_id':'2yTNGJu0CWKIms9vhpOPeY','genre':'Pop'},
{'user':'1288831866','playlist_id':'0lvwSeY2Og8XyileVyWZ1v','genre':'Pop'},
{'user':'dominic.rupp','playlist_id':'2zu4FqqzubZK4oXAZFEL4L','genre':'Pop'},
{'user':'redmusiccompany','playlist_id':'3U25AXqlklo3o6LqM1wX1I','genre':'Country'},
{'user':'1276364988','playlist_id':'7fIdjWhWzPbf1g5OSDKOXw','genre':'Country'},
{'user':'spotify','playlist_id':'37i9dQZF1DX13ZzXoot6Jc','genre':'Country'},
{'user':'spotify','playlist_id':'37i9dQZF1DX68H8ZujdnN7','genre':'Rock'},
{'user':'spotify','playlist_id':'37i9dQZF1DWZn9s1LNKPiM','genre':'Rock'},
{'user':'spotify','playlist_id':'37i9dQZF1DWWwzidNQX6jx','genre':'Rock'},
{'user':'sheffield806','playlist_id':'45vvbQl0GL7Niubb3H0U2Y','genre':'Rock'},
{'user':'redmusiccompany','playlist_id':'3yhmzZlOnRMCFV7PuMFPKy','genre':'Rock'},
{'user':'spotify','playlist_id':'37i9dQZF1DX12YIlzNkccL','genre':'Classical'},
{'user':'spotify','playlist_id':'37i9dQZF1DX8qqIDAkKiQg','genre':'Classical'},
{'user':'1269890947','playlist_id':'6Jn7zgkmA29infIRmptveP','genre':'Classical'},
{'user':'merovingin','playlist_id':'0ZRwrJ2EDGyKR6YgQPWXeO','genre':'Hip Hop'},
{'user':'akime1212','playlist_id':'2eY9lPvBKkluNSpeYJo71a','genre':'Hip Hop'},
{'user':'19jconnell79','playlist_id':'5772HGqmp2E99GQo5tfmcJ','genre':'Hip Hop'},
{'user':'1113721529','playlist_id':'0t3Mn6bXtPNWWZK6x9DtBH','genre':'Hip Hop'},
{'user':'playstation_music','playlist_id':'2NoR0KhNZ8oZnC3HoNt2FV','genre':'Hip Hop'},
{'user':'spotify','playlist_id':'37i9dQZF1DX8GjsySWIS1x','genre':'Gaming'},
{'user':'spotify','playlist_id':'37i9dQZF1DWYnY33e7En9e','genre':'Gaming'},
{'user':'1274440778','playlist_id':'5fdipI1K5g6ij4gCP8SvLj','genre':'Gaming'},
{'user':'spotify','playlist_id':'37i9dQZF1DX6AIf8KmeDe7','genre':'Gaming'},
{'user':'emma.ehrhardt','playlist_id':'0QApAL6CpODIMyweRdAvM4','genre':'Gaming'},
{'user':'spotify','playlist_id':'37i9dQZF1DX0SM0LYsmbMT','genre':'Jazz'},
{'user':'spotify','playlist_id':'37i9dQZF1DWVqfgj8NZEp1','genre':'Jazz'},
{'user':'spotify','playlist_id':'37i9dQZF1DX7YCknf2jT6s','genre':'Jazz'},
{'user':'spotify','playlist_id':'37i9dQZF1DWVzZlRWgqAGH','genre':'Jazz'},
{'user':'starbucks','playlist_id':'2UPjWBRjJqMQY3LRV2aDTS','genre':'Jazz'},
{'user':'spotify','playlist_id':'37i9dQZF1DWSiyIBdVQrkk','genre':'reggae'},
{'user':'boogalooboo','playlist_id':'4gRio043A2No9SRU7ERq6q','genre':'reggae'},
{'user':'constantinex','playlist_id':'0s8tfHVdQa8duFMev3ZlgD','genre':'reggae'},
{'user':'constantinex','playlist_id':'53iDeoyHodextygU8wpras','genre':'reggae'},
{'user':'jmelendez1024','playlist_id':'0ka9e6tQIfLXVSNXraZxNz','genre':'reggae'},
{'user':'spotify','playlist_id':'37i9dQZF1DX2ohL85TE8TI','genre':'K-pop'},
{'user':'spotify','playlist_id':'37i9dQZF1DXbirtHQBuwCo','genre':'K-pop'},
{'user':'spotify','playlist_id':'37i9dQZF1DX5eq3AONkaho','genre':'K-pop'},
{'user':'thehela1997','playlist_id':'4c1DULvTgCGWcTETJrra5A','genre':'K-pop'},
{'user':'12165116393','playlist_id':'3OWkXYSwYzKeN64mCqkouR','genre':'K-pop'}]

In [ ]:
playlistDF = pd.DataFrame(playListData)
print(playlistDF)

In [ ]:
playlistDF["playlist_id"][1]

In [ ]:
trackDFMain = pd.DataFrame()
for i in range(len(playlistDF)):
    resDF = pipeLineFunc(playlistDF["playlist_id"][i],playlistDF["user"][i],playlistDF["genre"][i])
    trackDFMain = trackDFMain.append(resDF)

In [ ]:
trackDFMain.to_csv('tracksInfo.csv')

In [ ]:
res = pipeLineFunc('37i9dQZF1DWTwnEm1IYyoj','spotify','Pop')
trackDFMain = trackDFMain.append(res)


In [ ]:
uniquetrackDF = trackDFMain.drop_duplicates('id')

In [ ]:
uniquetrackDF.to_csv('tracksMusicInfo.csv')

In [ ]:
spotifyClient.auth.token

In [14]:
uniquetrackDF = pd.read_csv('tracksMusicInfo.csv')

In [ ]:
uniquetrackDF['artist'][0]

In [ ]:
metadata = pygn.search(clientID=clientID, userID=userID, artist=uniquetrackDF['artist'][0], album=uniquetrackDF['album'][0], track=uniquetrackDF['track_name'][0])

In [ ]:
metadata['mood']

In [11]:
def getGNMood(art,alb,trk,trk_id):

    # art = str(inpdf['artist'])
    # alb =  str(inpdf['album'])
    # trk =  str(inpdf['track_name'])
    #print(art, alb, trk)
    mood_data = ""
    try:
        moodData = pygn.search(clientID=clientID,
                               userID=userID,
                               artist=art, 
                               album=alb,
                               track=trk)
        #print(moodData)
        
        if moodData['mood'] != {} or moodData['mood'] != None:
            if '1' in moodData['mood'].keys():
                if 'TEXT' in moodData['mood']['1'].keys():
                    mood_data = moodData['mood']['1']['TEXT']
        op = {'track_id':trk_id, 'mood':mood_data}
        return op
    except:
        print("error")
        op = {'track_id':trk_id, 'mood':mood_data}
        return op

In [12]:
def getAllMoodData(inpdf):
    opList = []
    for i in range(len(inpdf)):
        #print(inpdf.iloc[i,:])
        #print(inpdf['artist'][i])

        art = str(inpdf['artist'][i])
        alb =  str(inpdf['album'][i])
        trk =  str(inpdf['track_name'][i])
        trk_id =  str(inpdf['id'][i])
        print(i,art,alb,trk,trk_id)
        op = getGNMood(art,alb,trk,trk_id)
        opList.append(op)
    return opList

In [15]:
chunkList = list(range(0,len(uniquetrackDF),100))
dfList = []
for i in range(len(chunkList)):
    if i != (len(chunkList)-1):
        print(chunkList[i],chunkList[i+1]-1)
        tempDF = uniquetrackDF.iloc[chunkList[i]:chunkList[i+1]-1,:]
        dfList.append(tempDF)
    else:
        print(chunkList[i],len(uniquetrackDF)-1)
        tempDF = uniquetrackDF.iloc[chunkList[i]:,:]
        dfList.append(tempDF)

0 99
100 199
200 299
300 399
400 499
500 599
600 699
700 799
800 899
900 999
1000 1099
1100 1199
1200 1299
1300 1399
1400 1499
1500 1599
1600 1699
1700 1799
1800 1899
1900 1999
2000 2099
2100 2199
2200 2299
2300 2399
2400 2499
2500 2599
2600 2699
2700 2799
2800 2899
2900 2999
3000 3099
3100 3199
3200 3299
3300 3399
3400 3499
3500 3599
3600 3699
3700 3799
3800 3899
3900 3915


In [16]:
moodList = []


In [17]:
sub_start = pd.DataFrame(uniquetrackDF.iloc[:2000,:])
sub_2000 = pd.DataFrame(uniquetrackDF.iloc[2000:,:])

In [18]:
sub_2000 = sub_2000.reset_index(drop=True)

In [19]:

mood = getAllMoodData(sub_2000)
moodList = moodList + mood



0 Duo Kie 21 Centimetros Nosotros Lo Hicimos 65mRD6goKQX50reMDICnEx
1 Duo Kie 21 Centimetros Vamos Todos a Morir Aqui Dentro 0bJ4T1CGd4V4dchurLfKbV
2 Duo Kie Bomba Relógio Homer Simpson 2j3q1tNThMSstL8F0PN21r
3 Duo Kie 21 Centimetros Cualquiera Puede Tener un Mal Dia 06y6S7rDvikbaFiPiNFa3A
4 Duo Kie 21 Centimetros Dios Me Odia 6E6jJv9RGqlyGAaSkbZRZe
5 Piezas y Jayder Mal Ejemplo Espera Tras la Línea 15bj3k1EFKmQFW65OBIR98
6 Isusko&sbrv Marionetas Me Has Jodido el Día 4d49xL1EK220P8qWw4FnCS
7 Shotta Los Raperos Nunca Mueren Estaba Loca 501wNDrmMNNuR7i4BkgPnQ
8 Kultama The End Raíces 6KY90gnIK0rNAnMKHqLRUC
9 Nach Mejor Que El Silencio Hambre De Victoria 1bnj5APlJpy46DDgJD718P
10 SFDK Lista de invitados Lsd con Kultama 5JpSG4HPccviXoGAPkXzgE
11 SFDK Lista de invitados Cuando Todo Acabe con El Chojin - colabora Jefe De La M 0wZerLqhNKPlOBozNdztEh
12 Falsalarma Ley de Vida Fieles Con Lo Vivido 3h2ga3xeFDwm0bvY0TZKXh
13 Falsalarma Dramatica ¡A La Voz De Ya! 2ekK94k4zQaWMWrcAkYeDA
14 Falsalar

113 21 Savage Issa Album Baby Girl 6sBB5CBuzKJhDooro1IraE
114 NexXthursday Sway (feat. Quavo & Lil Yachty) Sway (feat. Quavo & Lil Yachty) 4QeBlVCl2KzUEcQZiuskRd
115 Bryson Tiller Get Mine Get Mine 4sxLB4ibrNZko2Y9wHfwZ3
116 Yogi Money On My Mind (feat. Juicy J & Alexander Lewis) Money On My Mind (feat. Juicy J & Alexander Lewis) 5gx4JbNdr7DSWQVkRw9UDK
117 Gorillaz Humanz (Deluxe) Ascension (feat. Vince Staples) 2bBMODHVTXGwE2aZYfbgCt
118 Tate Kobang Yeah Yeah 4msnrQPWcKwpONXNI06OJd
119 Tre Da Kid Run It (feat. TK Kravitz) Run It (feat. TK Kravitz) 6XHDrfPne3MtpmPlf2hR6G
120 Young Thug The Fate of the Furious: The Album Gang Up (with Young Thug, 2 Chainz & Wiz Khalifa feat. PnB Rock) 2rt6EWKD5iCod4cmWCCQKG
121 Denzel Curry Imperial Good Night 6bcnokzOVQxpn0quzHj6vc
122 Kid Ink F With U F With U 2kHdu1N31NkD5roBf1EHh7
123 T-Wayne Fell In Love (feat. XO) Fell In Love (feat. XO) 68S3G7OVjy5f1ufcky0DEW
124 DJ Cassidy Honor Honor 2LMCyYtrdA3qBX5GkKxDBf
125 Young Dolph King of Memphis Get Pa

200 C418 Minecraft - Volume Alpha Subwoofer Lullaby 1OJxI8lIWRqBvouJxW1nzN
201 Danny Elfman Avengers: Age of Ultron (Original Motion Picture Soundtrack) Heroes 3h8DQEl5vgt5BRIlVXXycl
202 Patrick Doyle Thor Sons of Odin 6x2fElXvwEvLoTTtDLrzjW
203 Dario Marianelli V For Vendetta: Music From The Motion Picture Evey Reborn 0NvKyH2zVnhLcgMM7wAuK6
204 Ramin Djawadi Game Of Thrones (Music From The HBO Series) Game Of Thrones 2kgJYQKyQ1CK5GWcCUGwwh
205 Hans Zimmer Pirates Of The Caribbean - Dead Man's Chest Original Soundtrack (English Version) Jack Sparrow 7lThyMeq8wY06ZeWrPwNrn
206 John Debney The Jungle Book (Original Motion Picture Soundtrack) The Man Village 1ZRXjiUrYkXLmOBatgpXDE
207 Giles Lamb Dead Island Trailer Theme (feat. Mairi Campbell, Peter Nicholson & Guido De Groot) - Single Dead Island Trailer Theme (feat. Mairi Campbell, Peter Nicholson & Guido De Groot) 1ku6iz80ht16xPwQNglXTv
208 Hans Zimmer The Dark Knight Rises Rise 3VdzKymHyCA7OQdUiDREKF
209 Howard Shore The Lord Of The R

280 Starship Knee Deep In The Hoopla We Built This City 7aHRctaQ7vjxVTVmY8OhAA
281 Frankie Goes To Hollywood Bang!... The Greatest Hits Of Frankie Goes To Hollywood Relax 3ugkTU1CHtrsjW70yQd66X
282 Montell Jordan This Is How We Do It This Is How We Do It 6uQKuonTU8VKBz5SHZuQXD
283 The Doobie Brothers The Captain And Me Long Train Runnin' 4nXkbcTj3nyww1cHkw5RAP
284 The Smashing Pumpkins Mellon Collie and the Infinite Sadness (2012 - Remaster) Bullet With Butterfly Wings 4qMzPtAZe0C9KWpWIzvZAP
285 Beastie Boys Ill Communication (Deluxe Version) [Remastered] Sabotage - 2009 Digital Remaster 7zLZFcyX6jN6YX7KFRzZf3
286 Tom Petty Greatest Hits Runnin' Down A Dream 128pBRZZgdi3BhS104k5Qv
287 KISS Double Platinum Detroit Rock City 2jz6rWGqVCYsymwxGoaON4
288 Smash Mouth Astro Lounge All Star 3cfOd4CMv2snFaKAnMdnvK
289 Sublime Sublime (Explicit Version) What I Got 3oVSsJXNADnYAnZWiEWFJH
290 Cutting Crew The Best Of Cutting Crew (I Just) Died in Your Arms 0YPXkQthLWrhNGoKTbwCJ8
291 Don Henley The

378 Kaskade Atmosphere Last Chance 5bw3ANysdLNdRl8wZfunVp
379 deadmau5 Strobe Strobe - Radio Edit 6c9EGVj5CaOeoKd9ecMW1U
380 Martin Solveig Tomorrowland 2012_02 The Night Out - A-Trak vs. Martin Rework 3Dx4tf1PskkT1jYqnWjBKr
381 Phillip Alpha Enhanced Music - Autumn Collection 2010 Valente - Estiva Remix 4rl62nhAXeN2Yih7yTl1Xw
382 Emma Hewitt Rewind Rewind - Mikkas Remix 6ZCdcFLHmRk4NXylGaIHPk
383 Antillas Damaged Damaged - Radio Mix 7GMTLbNJXncUbewD6VOAFh
384 Feint Feint EP1 Reprise - Original Mix 411rcI6KFOx1BYw8HY4shd
385 Jessie J Nobody's Perfect Nobody's Perfect - Netsky Full Vocal Remix 1POpzXwi5LRsjwWO9sgRAj
386 Zomboy The Dead Symphonic EP Nuclear (Hands Up) 2ThslNnPGmK82KFcCF2Lxm
387 Krewella Play Hard EP Alive 2SHnUyZq0zwmvRIl4WY77G
388 Avicii TRUE Addicted To You 0xIVvRmjztR1AwuHrkhH41
389 Avicii TRUE You Make Me 7HC3sppue0re7HPfo5zscF
390 Avicii TRUE All You Need Is Love 7oSlMkRv5AhPnnGWdFTv20
391 Deniz Koyu Tung! Tung! 3zaj7MGWMwHZ5eZvKcuaGz
392 Andrew Rayel Universal Reli

...

1456 Arcangel El Fenomeno Chica Virtual - Bonus 3x6La5Wg0hOsQXcuqtqxZU
1457 R.K.M & Ken-Y Masterpiece Dame Lo Que Quiero 4bOPAT1KcuUrwt111lhxzC
1458 R.K.M & Ken-Y Masterpiece Igual Que Ayer 7qwmzs8N2HRYbqNzybkU75
1459 R.K.M & Ken-Y 2008 Año De Exitos Reggaeton Urbano Me Matas 6GrPaW2OIHuweYcVRoYbmv
1460 R.K.M & Ken-Y Reggaeton Simply The Best Down 09QZbXkrHRww7Epv7qSnL0
1461 Toby Love Toby Love Tengo Un Amor - Remix Version 4mDWapOAzoFsHu8TPcP2R0
1462 R.K.M & Ken-Y Alex Gargolas Greatest Hits Un Sueño 0dA58RZci59FbrNbNbhGJj
1463 R.K.M & Ken-Y Masterpiece Commemorative Edition Down - Remix 58eejL9t76Ox6LqkaLLy6X
1464 Alexis y Fido Sobrenatural 5 Letras 0tDSgSmZsbxCkdkfUPjg59
1465 Alexis y Fido The Pitbulls Eso Ehh..!!! 7uCZiUZqnKSGBnr3YZTcxA
1466 Alexis y Fido The Pitbulls El Tiburón 4H0vsbZL1DlKDKLrHHNaqb
1467 Yaviah La Mision 4 Wiki Wiki 4TofHskDzd6pLLiAqmUjvg
1468 Las Guanabanas Guillaera Mi Gatita Y Yo 0luIn6z2iePC9DdbNHxX5y
1469 Las Guanabanas Guillaera Vamos Pa' La Disco 5jGESibdJ

1554 Kim Yeonji Grand Prince, Pt. 1 (Original Television Soundtrack) Follow the Road 5ayUn7feR15HnrIBXGxlvO
error
1555 태하 Cross (Original Television Soundtrack), Pt. 5 I Need You and I Want You 1AHARuWaGWxU0stG8GeZAd
error
1556 Gonne Choi Cross (Original Television Soundtrack), Pt. 3 Childhood 69swYUmQXk3DIk3lCHICXz
1557 Wheesung 키스 먼저 할까요? Should We Kiss First? (Original Television Soundtrack), Pt. 4 독백 Monologue 1ie4ezIx6Zwp1YzJjwxYC2
1558 YESUNG 키스 먼저 할까요? Should We Kiss First? (Original Television Soundtrack), Pt. 5 혼자 하는 일 On My Own 5qo81amE7v6JBbTDQ2MrZo
1559 CHEN Moonlovers: Scarlet Heart Ryeo (Original Television Soundtrack), Pt 1 For You 7rLvsAO1yb7ElxPhkz60qh
1560 담소네공방 로봇이 아니야 Pt. 4 Original Television Soundtrack 마음 다해 사랑하는 일 0fxeARRHfVIvvXzClAtks0
error
1561 스텔라장 로봇이 아니야, (Original Television Soundtrack) 날 알아줄까 4TffylCBRBcMZQvCnqUIKK
1562 J_ust First Kiss (Original Soundtrack) Into You 6kZbHiG44E9HcD1Dq9AhNS
1563 Leesa A Korean Odyssey (Original Television Soundtrack), Pt. 

1651 Yang Yoseob YANG YOSEOP 2ND MINI ALBUM ‘白’ Where I am gone 6uCmZdO7HFRo12LTH1qlNn
1652 B.O. COMFORT Sunday In My Bedroom (Remix) 0nClwn3MZbYCqOcjK7nYdi
1653 DEAN What 2 Do What 2 Do 4dXlF5EB21Zes3aTWS2DFF
error
1654 Zion.T Prison Playbook (Original Television Soundtrack), Pt. 9 Those Days (without you) 7uQ0QBWGiVsfWcWxikYsWV
1655 Reddy Think (Feat. Jay Park) Think (Feat. Jay Park) 0YBLAFPF2nYumXT2PGEnLe
1656 Penomeco L.I.E (Prod. ZICO) L.I.E - Prod. ZICO 7jwEER43W8bYYsKUdl39Aa
error
1657 Nuz 30 island 5H8Ticj6zwi5TRkLR01trN
1658 DEAN Limbo Come Over 1ieDhJfetL1wUFTC0YIriP
1659 D.O D.O.P.E. Falling Down 4Xq8fPbkEcmBT5jMBn7iu7
1660 Jeebanoff KARMA Timid 6Stb5xQgTF3bzGWlqryMxG
1661 Rico Fruit Juice Fruit Juice 5z9KRAObGxrvKW64vNxsfw
error
1662 JEONG SEWOON THE 1ST MINI ALBUM PART.2 `AFTER` Toc, toC! (PROD. MAKTUB) 4OK4tHSUnCXpBfIusCOdAo
1663 HEIZE Don't Come Back Don't Come Back(feat. Yong Jun Hyung) 5eYNVMkqUDaZMGSXhwaO2O
error
1664 Paul Kim Paul Kim The First Album Part. 2 ‘Tunnel’

1767 Crush 잊어버리지마 Don't Forget (feat. 태연 Taeyeon) - Single 잊어버리지마 Don't Forget (feat. 태연 Taeyeon) 2HC6c1d7SiznMV4rkmmxBB
1768 B.A.P NOIR SKYDIVE 54aUpI8yA41tK4zh1fhTr6
error
1769 Suzy Dream Dream 3JBnDOUd18QKjDqSYuOfpm
error
1770 Block B - BASTARZ WELCOME 2 BASTARZ Selfish & Beautiful Girl 15KWegygpWy0lE4GISSgmV
1771 Urban Zakapa Still I Don't Love You 7neOIVKDsLaUXV5y84jGuY
1772 Crush Wonderlust - EP Fall 0Ft9pLHK8GSYxBjDLXZBMd
1773 Henry 우리 둘 Runnin' 우리 둘 Runnin' 68oQulR4Fh2Rqd4s1mXcGh
1774 Alesso Years Years - Korean Version 2OL2FYLPYcp5nW89MUGqmS
error
1775 SECHSKIES THREE WORDS THREE WORDS 1gyuzdxaZMhKt6rvUJ3keI
error
1776 C Jamm, BewhY puzzle puzzle 3kJjYTtOQPiVSr1UNEqM0u
1777 Simon Dominic SHOW ME THE MONEY 5 Episode 2 Comfortable 2lEzKOtt7q64jQZ5meZsY8
1778 Lyn 태양의 후예 (Original Television Soundtrack), Pt. 7 With You 3VzgQ4i6s4ryHXf3kOHKLy
error
1779 BTS FACE YOURSELF Let Go 1OqDvYVDhJyFZny7XlfIyZ
error
1780 SUPER JUNIOR REPLAY - The 8th Repackage Album Lo Siento 2VVqxQhALecXvaA

error
1887 SUPER JUNIOR Mr. Simple Mr. Simple 1EBA71Up4r13q6ggBCwBdJ
error
1888 G-DRAGON One of a Kind Crayon 4CluCFFy6i1MUL1cYdoCif
error
1889 HyunA A'wesome How's This 0Zb4u1riI2E30ay5pVYAUW
error
1890 iKON WELCOME BACK RHYTHM TA 374WyJeaSR9Cmq4kuFzoqu
error
1891 BTS The Most Beautiful Moment in Life Pt.2 Silver Spoon 1ACMf8SOANvKlOu4vTWiJN
error
1892 BTS 2 Cool 4 Skool No More Dream 4frlkLwlnm58ejpNwWjxyl
1893 I.O.I Whatta Man Whatta Man (Good man) 2MhEeappm9nGPv1p7a1YMS
error
1894 Girls' Generation THE BEST Run Devil Run - Dance ver. 6eprmDJUM2tme83k1Dbb8j
error
1895 BOBBY LOVE AND FALL I LOVE YOU 1tSHQvRQD0RQHdQjmEfrCv
error
1896 Wanna One 1X1=1(TO BE ONE) Burn It Up 0WJVy5IYA9AeAI1htZDq0C
1897 Monsta X THE CLAN pt.2.5 [BEAUTIFUL] Beautiful 3utoyXrzB3d8ahf4tqazTs
error
1898 SEVENTEEN SEVENTEEN 2nd Mini Album 'Boys Be' Mansae 5rIsrawNyPv1q8v11iwJsh
1899 4Minute Name is 4minute What's your name? 13xfTHyEclnBd7XAS1P5Er
error
1900 Teen Top TEEN TOP CLASS Rocking 1su62pO1ShAp7F4JlY5vuo

In [ ]:
moodList_BK  = moodList

In [20]:
a = {"mood2":moodList}
with open('mood2.pickle', 'wb') as handle:
    pickle.dump(a, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [21]:
with open('mood1.pickle', 'rb') as handle:
    moodList1 = pickle.load(handle)

In [24]:
moodListMain = moodList1['mood1']

In [27]:
a = {"moodMain":moodListMain}
with open('moodMain.pickle', 'wb') as handle:
    pickle.dump(a, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [25]:
moodListMain = moodListMain + moodList

In [26]:
len(moodListMain)

3916

In [28]:
# merge
moodDF = pd.DataFrame(moodListMain)

In [29]:
moodDF

,mood,track_id
0,Brooding,1NrJYpdAi7uosDRPmSYrsG
1,Yearning,7evPpWWu01Klxi0KlPxR17
2,Empowering,0CZ8lquoTX2Dkg7Ak2inwA
3,Gritty,1bhUWB0zJMIKr9yVPrkEuI
4,Yearning,5Db9VIdDsN5yu3Eu7CT0i4
5,Brooding,63SevszngYpZOwf63o61K4
6,Empowering,11bD1JtSjlIgKgZG2134DZ
7,Yearning,3CKCZ9pfwAfoMZlMncA1Nc
8,,1HNkqx9Ahdgi1Ixy2xkKkL
9,Sensual,2YlZnw2ikdb837oKMKjBkW


In [ ]:
listone = [{"1":1},2,3]
listtwo = [4,5,6]

mergedlist = listone + listtwo
mergedlist

In [ ]:
moodList_BK

In [ ]:
uniquetrackDF.iloc[:9,:]

In [31]:
mainDF = pd.merge(uniquetrackDF, moodDF, how='left', left_on=['id'], right_on=['track_id'])

In [35]:
mainDF

,album,artist,genre,id,track_name,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,mood,track_id
0,Dreaming Out Loud,OneRepublic,Pop,1NrJYpdAi7uosDRPmSYrsG,Apologize,0.348000,0.591,208107,0.718,0.000118,8,0.1070,-6.025,1,0.0368,117.995,4,0.4680,Brooding,1NrJYpdAi7uosDRPmSYrsG
1,Illuminate (Deluxe),Shawn Mendes,Pop,7evPpWWu01Klxi0KlPxR17,Mercy,0.133000,0.568,208733,0.686,0.000000,11,0.1100,-4.901,0,0.0903,148.294,4,0.4000,Yearning,7evPpWWu01Klxi0KlPxR17
2,1950,King Princess,Pop,0CZ8lquoTX2Dkg7Ak2inwA,1950,0.648000,0.600,225133,0.535,0.000005,4,0.1620,-7.043,1,0.1070,71.912,4,0.2690,Empowering,0CZ8lquoTX2Dkg7Ak2inwA
3,Perfect Duet (Ed Sheeran & Beyoncé),Ed Sheeran,Pop,1bhUWB0zJMIKr9yVPrkEuI,Perfect Duet (Ed Sheeran & Beyoncé),0.779000,0.587,259550,0.299,0.000000,8,0.1230,-7.365,1,0.0263,94.992,3,0.3560,Gritty,1bhUWB0zJMIKr9yVPrkEuI
4,In The Lonely Hour,Sam Smith,Pop,5Db9VIdDsN5yu3Eu7CT0i4,Stay With Me,0.568000,0.479,172724,0.419,0.000217,0,0.1100,-6.517,1,0.0389,85.014,4,0.1860,Yearning,5Db9VIdDsN5yu3Eu7CT0i4
5,Nevermind,Dennis Lloyd,Pop,63SevszngYpZOwf63o61K4,Nevermind,0.159000,0.601,156600,0.688,0.000013,5,0.4140,-8.374,0,0.2020,99.933,4,0.0793,Brooding,63SevszngYpZOwf63o61K4
6,Eyes Open,Snow Patrol,Pop,11bD1JtSjlIgKgZG2134DZ,Chasing Cars,0.193000,0.557,267933,0.609,0.000077,9,0.1350,-4.463,1,0.0259,104.010,4,0.1720,Empowering,11bD1JtSjlIgKgZG2134DZ
7,21,Adele,Pop,3CKCZ9pfwAfoMZlMncA1Nc,Set Fire to the Rain,0.003820,0.610,241693,0.683,0.000002,2,0.1250,-3.879,0,0.0253,108.003,4,0.4660,Yearning,3CKCZ9pfwAfoMZlMncA1Nc
8,x (Deluxe Edition),Ed Sheeran,Pop,1HNkqx9Ahdgi1Ixy2xkKkL,Photograph,0.607000,0.614,258987,0.379,0.000464,4,0.0986,-10.480,1,0.0476,107.989,4,0.2010,,1HNkqx9Ahdgi1Ixy2xkKkL
9,Title (Deluxe),Meghan Trainor,Pop,2YlZnw2ikdb837oKMKjBkW,Like I'm Gonna Lose You,0.400000,0.630,225053,0.530,0.000000,0,0.1770,-7.259,1,0.0434,108.038,3,0.4170,Sensual,2YlZnw2ikdb837oKMKjBkW
